In [1]:
%%capture
from re import sub
from pandas import DataFrame, ExcelWriter
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from plotnine import *

from sys import path 
path.insert(0, '../core/')

from agyw import AgywPrev, AgywPrevCommune
from active import actif_served as AGYW_ACTIF

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

date_du_jour = datetime.today().strftime("%d_%m_%Y")


OperationalError: (pymysql.err.OperationalError) (1248, 'Every derived table must have its own alias')
[SQL: 
SELECT 
    a.id_patient,
    h.id_parenting_group,
    g.departement,
    g.commune,
    b.nbre_pres_for_inter,
    h.nbre_parenting_coupe_present,
    b.has_comdom_topic,
    b.has_preventive_vbg,
    d.number_of_condoms_sensibilize,
    d.number_condoms_sensibilization_date_in_the_interval,
    d.number_condoms_reception_in_the_interval,
    d.number_hiv_test_awareness_date_in_the_interval,
    d.type_of_test_vih,
    d.number_autotest_date_in_the_interval,
    d.number_test_date_in_the_interval,
    d.test_results,
    autotest_result,
    d.number_hiv_treatment_start_date_in_the_interval,
    d.number_vbg_treatment_date_in_the_interval,
    d.number_gynecological_care_date_in_the_interval,
    d.number_prep_awareness_date_in_the_interval,
    d.number_prep_reference_date_in_the_interval,
    d.number_prep_initiation_date_in_the_interval,
    d.number_contraceptive_sensibil

In [ ]:
from pandas import DataFrame


In [ ]:
%%capture
datim = AgywPrev()


In [ ]:
datim.total_mastersheet
datim.total_dreams_valid
datim.who_am_i
datim.total_datimI
datim.total_datimII
datim.total_datimIII
datim.total_datimIV
datim.total_datim_general
AgywPrev.datim_titleI()
datim.datim_agyw_prevI()
AgywPrev.datim_titleII()
datim.datim_agyw_prevII()
AgywPrev.datim_titleIII()
datim.datim_agyw_prevIII()
AgywPrev.datim_titleIV()
datim.datim_agyw_prevIV()
datim.datim_vital_info()


10047

9755

'DATIM'

4497

4598

...

'Number of active DREAMS participants that have fully completed at least one DREAMS service/intervention but NOT the full primary package of services/interventions as of the past 6 months at Q2 or the past 12 months at Q4.'

,Time/Age/Sex,10-14,15-19,20-24,25-29
1,0-6 months,37,27,16,0
2,07-12 months,68,68,25,0
3,13-24 months,28,44,5,0
4,25+ months,38,86,102,0
5,Total,171,225,148,0


'Number of active DREAMS participants that have started a DREAMS service but have not yet completed it in the past 6 months at Q2 or 12 months at Q4.'

,Time/Age/Sex,10-14,15-19,20-24,25-29
1,0-6 months,24,29,14,0
2,07-12 months,4,10,3,0
3,13-24 months,7,19,5,0
4,25+ months,0,1,0,0
5,Total,35,59,22,0


,Number of active DREAMS participants that received an evidence-based intervention focused on preventing violence within the reporting period.,9095
1,Number of active DREAMS participants that rece...,366
2,Number of active DREAMS participants that comp...,954


In [ ]:
%%capture
def name_handler(s):
    s = sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespace with a single dash
    s = sub(r"\s+", '_', s)
    return s

datimcommunes = []
cols  = datim.data_mastersheet.commune.unique()
columns = [i for i in cols if i is not None]
for columns in columns:
    globals()[f"datim_{name_handler(columns)}"] = AgywPrevCommune(f"{columns}")
    datimcommunes.append(globals().get(f"datim_{name_handler(columns)}"))


for dcommunes in datimcommunes:
  dcommunes.total_mastersheet 
  dcommunes.total_dreams_valid
  dcommunes.who_am_i
  dcommunes.total_datimI
  dcommunes.total_datimII
  dcommunes.total_datimIII
  dcommunes.total_datimIV
  dcommunes.total_datim_general
  AgywPrevCommune.datim_titleI()
  dcommunes.datim_agyw_prevI()
  AgywPrevCommune.datim_titleII()
  dcommunes.datim_agyw_prevII()
  AgywPrevCommune.datim_titleIII()
  dcommunes.datim_agyw_prevIII()
  AgywPrevCommune.datim_titleIV()
  dcommunes.datim_agyw_prevIV()
  print(" ")
  print(" ")
  print(" ")
  print(" ")


In [ ]:
datim


<AgywPrev DATIM>

In [ ]:


""" p = (ggplot(stack_data,aes(x="supp",y="data",fill="label",width=0.7)) 
 + geom_bar(position=position_stack(vjust=0.5),stat="identity")
 + geom_text(aes(label="data"),position=position_stack(vjust=0.5))
 + ggtitle(" Les composantes AGYW PREV face au Target")
 + xlab("")
 +ylab("")
 +labs(caption =  "HIVHaiti")
 + theme_seaborn()
)
p
 """
 #ggsave(plot=p,filename="composante_agywprev.png",dpi=1000)


' p = (ggplot(stack_data,aes(x="supp",y="data",fill="label",width=0.7)) \n + geom_bar(position=position_stack(vjust=0.5),stat="identity")\n + geom_text(aes(label="data"),position=position_stack(vjust=0.5))\n + ggtitle(" Les composantes AGYW PREV face au Target")\n + xlab("")\n +ylab("")\n +labs(caption =  "HIVHaiti")\n + theme_seaborn()\n)\np\n '

In [ ]:
agyw_per_commune = datim.data_dreams_valid.pivot_table(
    values="id_patient",
    index="commune",
    columns="age_range",
    aggfunc="count",
    fill_value=0,
    margins=True,
    margins_name="Total"
)[:-1].sort_values("Total",ascending=False)
agyw_per_commune


age_range,10-14,15-19,20-24,Total
commune,,,,
Kenscoff,1083,609,144,1836
Saint-Marc,515,895,407,1817
Verrettes,349,395,164,908
Carrefour,279,329,192,800
Gressier,178,234,114,526
Dessalines,324,138,23,485
Cap-Haïtien,169,220,65,454
Grande Saline,132,171,129,432
Port-au-Prince,123,159,136,418


In [ ]:
usaid_verification = ExcelWriter(f"./tables_verification/datim_verification.xlsx",engine="xlsxwriter")
agyw_per_commune.to_excel(usaid_verification,sheet_name="agyw_per_commune",index=True,na_rep="")
datim.data_agyw_prevI.to_excel(usaid_verification,sheet_name="composante I",index=False,na_rep="")
datim.data_agyw_prevII.to_excel(usaid_verification,sheet_name="composante II",index=False,na_rep="")
datim.data_agyw_prevIII.to_excel(usaid_verification,sheet_name="Composante III",index=False,na_rep="")
datim.data_agyw_prevIV.to_excel(usaid_verification,sheet_name="Composante IV",index=False,na_rep="")
datim.data_dreams_valid.to_excel(usaid_verification,sheet_name="table_global",index=False,na_rep="")
usaid_verification.save()

""" usaid_verification = ExcelWriter(f"./tables_verification/datim_verification_{date_du_jour}.xlsx",engine="xlsxwriter")
datim.data_agyw_prevI.to_excel(usaid_verification,sheet_name="composante I",index=False,na_rep="")
datim.data_agyw_prevII.to_excel(usaid_verification,sheet_name="composante II",index=False,na_rep="")
datim.data_agyw_prevIII.to_excel(usaid_verification,sheet_name="Composante III",index=False,na_rep="")
datim.data_agyw_prevIV.to_excel(usaid_verification,sheet_name="Composante IV",index=False,na_rep="")
datim.data_dreams_valid.to_excel(usaid_verification,sheet_name="table_global",index=False,na_rep="")
usaid_verification.save()
 """


""" usaid_verification = ExcelWriter(f"./tables_verification/datim_verification_Q2FY23_{date_du_jour}.xlsx",engine="xlsxwriter")
agyw_per_commune.to_excel(usaid_verification,sheet_name="agyw_per_commune",index=True,na_rep="")
datim.data_agyw_prevI.to_excel(usaid_verification,sheet_name="composante I",index=False,na_rep="")
datim.data_agyw_prevII.to_excel(usaid_verification,sheet_name="composante II",index=False,na_rep="")
datim.data_agyw_prevIII.to_excel(usaid_verification,sheet_name="Composante III",index=False,na_rep="")
datim.data_agyw_prevIV.to_excel(usaid_verification,sheet_name="Composante IV",index=False,na_rep="")
datim.data_dreams_valid.to_excel(usaid_verification,sheet_name="table_global",index=False,na_rep="")
usaid_verification.save() """

""" usaid_verification = ExcelWriter(f"./tables_verification/ONFY21/datim_verification.xlsx",engine="xlsxwriter")
datim.data_agyw_prevI.to_excel(usaid_verification,sheet_name="composante I",index=False,na_rep="")
datim.data_agyw_prevII.to_excel(usaid_verification,sheet_name="composante II",index=False,na_rep="")
datim.data_agyw_prevIII.to_excel(usaid_verification,sheet_name="Composante III",index=False,na_rep="")
datim.data_agyw_prevIV.to_excel(usaid_verification,sheet_name="Composante IV",index=False,na_rep="")
datim.data_dreams_valid.to_excel(usaid_verification,sheet_name="table_global",index=False,na_rep="")
usaid_verification.save() """

""" usaid_verification = ExcelWriter(f"./tables_verification/Q3/datim_verification_{date_du_jour}.xlsx",engine="xlsxwriter")
datim.data_agyw_prevI.to_excel(usaid_verification,sheet_name="composante I",index=False,na_rep="")
datim.data_agyw_prevII.to_excel(usaid_verification,sheet_name="composante II",index=False,na_rep="")
datim.data_agyw_prevIII.to_excel(usaid_verification,sheet_name="Composante III",index=False,na_rep="")
datim.data_agyw_prevIV.to_excel(usaid_verification,sheet_name="Composante IV",index=False,na_rep="")
datim.data_dreams_valid.to_excel(usaid_verification,sheet_name="table_global",index=False,na_rep="")
usaid_verification.save() """ 

""" usaid_verification = ExcelWriter(f"./tables_verification/Q2/datim_verification_{date_du_jour}.xlsx",engine="xlsxwriter")
datim.data_agyw_prevI.to_excel(usaid_verification,sheet_name="composante I",index=False,na_rep="")
datim.data_agyw_prevII.to_excel(usaid_verification,sheet_name="composante II",index=False,na_rep="")
datim.data_agyw_prevIII.to_excel(usaid_verification,sheet_name="Composante III",index=False,na_rep="")
datim.data_agyw_prevIV.to_excel(usaid_verification,sheet_name="Composante IV",index=False,na_rep="")
datim.data_dreams_valid.to_excel(usaid_verification,sheet_name="table_global",index=False,na_rep="")
usaid_verification.save() """


""" usaid_verification = ExcelWriter(f"./tables_verification/special/datim_verification_{date_du_jour}.xlsx",engine="xlsxwriter")
datim.data_agyw_prevI.to_excel(usaid_verification,sheet_name="composante I",index=False,na_rep="")
datim.data_agyw_prevII.to_excel(usaid_verification,sheet_name="composante II",index=False,na_rep="")
datim.data_agyw_prevIII.to_excel(usaid_verification,sheet_name="Composante III",index=False,na_rep="")
datim.data_agyw_prevIV.to_excel(usaid_verification,sheet_name="Composante IV",index=False,na_rep="")
datim.data_dreams_valid.to_excel(usaid_verification,sheet_name="table_global",index=False,na_rep="")
usaid_verification.save() """  


/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_20449/644804739.py:8: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version


' usaid_verification = ExcelWriter(f"./tables_verification/datim_verification_{date_du_jour}.xlsx",engine="xlsxwriter")\ndatim.data_agyw_prevI.to_excel(usaid_verification,sheet_name="composante I",index=False,na_rep="")\ndatim.data_agyw_prevII.to_excel(usaid_verification,sheet_name="composante II",index=False,na_rep="")\ndatim.data_agyw_prevIII.to_excel(usaid_verification,sheet_name="Composante III",index=False,na_rep="")\ndatim.data_agyw_prevIV.to_excel(usaid_verification,sheet_name="Composante IV",index=False,na_rep="")\ndatim.data_dreams_valid.to_excel(usaid_verification,sheet_name="table_global",index=False,na_rep="")\nusaid_verification.save()\n '

' usaid_verification = ExcelWriter(f"./tables_verification/datim_verification_Q2FY23_{date_du_jour}.xlsx",engine="xlsxwriter")\nagyw_per_commune.to_excel(usaid_verification,sheet_name="agyw_per_commune",index=True,na_rep="")\ndatim.data_agyw_prevI.to_excel(usaid_verification,sheet_name="composante I",index=False,na_rep="")\ndatim.data_agyw_prevII.to_excel(usaid_verification,sheet_name="composante II",index=False,na_rep="")\ndatim.data_agyw_prevIII.to_excel(usaid_verification,sheet_name="Composante III",index=False,na_rep="")\ndatim.data_agyw_prevIV.to_excel(usaid_verification,sheet_name="Composante IV",index=False,na_rep="")\ndatim.data_dreams_valid.to_excel(usaid_verification,sheet_name="table_global",index=False,na_rep="")\nusaid_verification.save() '

' usaid_verification = ExcelWriter(f"./tables_verification/ONFY21/datim_verification.xlsx",engine="xlsxwriter")\ndatim.data_agyw_prevI.to_excel(usaid_verification,sheet_name="composante I",index=False,na_rep="")\ndatim.data_agyw_prevII.to_excel(usaid_verification,sheet_name="composante II",index=False,na_rep="")\ndatim.data_agyw_prevIII.to_excel(usaid_verification,sheet_name="Composante III",index=False,na_rep="")\ndatim.data_agyw_prevIV.to_excel(usaid_verification,sheet_name="Composante IV",index=False,na_rep="")\ndatim.data_dreams_valid.to_excel(usaid_verification,sheet_name="table_global",index=False,na_rep="")\nusaid_verification.save() '

' usaid_verification = ExcelWriter(f"./tables_verification/Q3/datim_verification_{date_du_jour}.xlsx",engine="xlsxwriter")\ndatim.data_agyw_prevI.to_excel(usaid_verification,sheet_name="composante I",index=False,na_rep="")\ndatim.data_agyw_prevII.to_excel(usaid_verification,sheet_name="composante II",index=False,na_rep="")\ndatim.data_agyw_prevIII.to_excel(usaid_verification,sheet_name="Composante III",index=False,na_rep="")\ndatim.data_agyw_prevIV.to_excel(usaid_verification,sheet_name="Composante IV",index=False,na_rep="")\ndatim.data_dreams_valid.to_excel(usaid_verification,sheet_name="table_global",index=False,na_rep="")\nusaid_verification.save() '

' usaid_verification = ExcelWriter(f"./tables_verification/Q2/datim_verification_{date_du_jour}.xlsx",engine="xlsxwriter")\ndatim.data_agyw_prevI.to_excel(usaid_verification,sheet_name="composante I",index=False,na_rep="")\ndatim.data_agyw_prevII.to_excel(usaid_verification,sheet_name="composante II",index=False,na_rep="")\ndatim.data_agyw_prevIII.to_excel(usaid_verification,sheet_name="Composante III",index=False,na_rep="")\ndatim.data_agyw_prevIV.to_excel(usaid_verification,sheet_name="Composante IV",index=False,na_rep="")\ndatim.data_dreams_valid.to_excel(usaid_verification,sheet_name="table_global",index=False,na_rep="")\nusaid_verification.save() '

' usaid_verification = ExcelWriter(f"./tables_verification/special/datim_verification_{date_du_jour}.xlsx",engine="xlsxwriter")\ndatim.data_agyw_prevI.to_excel(usaid_verification,sheet_name="composante I",index=False,na_rep="")\ndatim.data_agyw_prevII.to_excel(usaid_verification,sheet_name="composante II",index=False,na_rep="")\ndatim.data_agyw_prevIII.to_excel(usaid_verification,sheet_name="Composante III",index=False,na_rep="")\ndatim.data_agyw_prevIV.to_excel(usaid_verification,sheet_name="Composante IV",index=False,na_rep="")\ndatim.data_dreams_valid.to_excel(usaid_verification,sheet_name="table_global",index=False,na_rep="")\nusaid_verification.save() '

# narrative based information

In [ ]:
from pandas import DataFrame, read_sql_query, Int32Dtype, read_excel
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
import pymysql
from sqlalchemy import create_engine, text
from decouple import config 
from dotenv import load_dotenv
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = f"""
SELECT
    dh.office,
    d.case_id,
    dm.id_patient AS id_patient,
    p.patient_code AS code,
    d.a_non_patisipan_an AS first_name,
    d.b_siyati AS last_name,
    TIMESTAMPDIFF(YEAR,
        d.nan_ki_dat_ou_fet,
        NOW()) AS age,
    IF(TIMESTAMPDIFF(YEAR,
            d.nan_ki_dat_ou_fet,
            NOW()) >= 10
            AND TIMESTAMPDIFF(YEAR,
            d.nan_ki_dat_ou_fet,
            NOW()) <= 14,
        '10-14',
        IF(TIMESTAMPDIFF(YEAR,
                d.nan_ki_dat_ou_fet,
                NOW()) >= 15
                AND TIMESTAMPDIFF(YEAR,
                d.nan_ki_dat_ou_fet,
                NOW()) <= 19,
            '15-19',
            IF(TIMESTAMPDIFF(YEAR,
                    d.nan_ki_dat_ou_fet,
                    NOW()) >= 20
                    AND TIMESTAMPDIFF(YEAR,
                    d.nan_ki_dat_ou_fet,
                    NOW()) <= 24,
                '20-24',
                IF(TIMESTAMPDIFF(YEAR,
                        d.nan_ki_dat_ou_fet,
                        NOW()) >= 25
                        AND TIMESTAMPDIFF(YEAR,
                        d.nan_ki_dat_ou_fet,
                        NOW()) <= 29,
                    '25-29',
                    'not_valid_age')))) AS age_range,
    IF(TIMESTAMPDIFF(YEAR,
            d.nan_ki_dat_ou_fet,
            NOW()) >= 10
            AND TIMESTAMPDIFF(YEAR,
            d.nan_ki_dat_ou_fet,
            NOW()) <= 14,
        '10-14',
        IF(TIMESTAMPDIFF(YEAR,
                d.nan_ki_dat_ou_fet,
                NOW()) >= 15
                AND TIMESTAMPDIFF(YEAR,
                d.nan_ki_dat_ou_fet,
                NOW()) <= 17,
            '15-17',
            IF(TIMESTAMPDIFF(YEAR,
                    d.nan_ki_dat_ou_fet,
                    NOW()) >= 18
                    AND TIMESTAMPDIFF(YEAR,
                    d.nan_ki_dat_ou_fet,
                    NOW()) <= 24,
                '18-24',
                IF(TIMESTAMPDIFF(YEAR,
                        d.nan_ki_dat_ou_fet,
                        NOW()) >= 25
                        AND TIMESTAMPDIFF(YEAR,
                        d.nan_ki_dat_ou_fet,
                        NOW()) <= 29,
                    '25-29',
                    'not_valid_age')))) AS ovc_age,
    d.nan_ki_dat_ou_fet AS dob,
    IF(TIMESTAMPDIFF(MONTH,
            d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817,
            NOW()) >= 0
            AND TIMESTAMPDIFF(MONTH,
            d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817,
            NOW()) <= 6,
        '0-6 months',
        IF(TIMESTAMPDIFF(MONTH,
                d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817,
                NOW()) >= 7
                AND TIMESTAMPDIFF(MONTH,
                d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817,
                NOW()) <= 12,
            '07-12 months',
            IF(TIMESTAMPDIFF(MONTH,
                    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817,
                    NOW()) >= 13
                    AND TIMESTAMPDIFF(MONTH,
                    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817,
                    NOW()) <= 24,
                '13-24 months',
                '25+ months'))) AS month_in_program_range,
    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS interview_date,
    d.e__telefn,
    d.d_adrs AS adress,
    IF(dm.id IS NOT NULL, 'yes', 'no') AS already_in_a_group,
    dm.id_group AS actual_id_group,
    dg.name AS actual_group_name,
    dm.id_parenting_group AS actual_id_parenting_group,
    dpg.name AS actual_parenting_group_name,
    dh.name AS actual_hub,
    dh.id as actual_id_hub,
    ld.name AS actual_departement,
    d.f_komin AS commune,
    d.g_seksyon_kominal AS commune_section,
    d.b1_non_moun_mennen_entvyou_a AS interviewer_firstname,
    d.c1_siyati_moun_ki_f_entvyou_a AS interviewer_lastname,
    d.d1_kad AS interviewer_role,
    d.lot_kad AS interviewer_other_info,
    d.h_kote_entvyou_a_ft AS interview_location,
    d.paran_ou_vivan AS is_your_parent_alive,
    d.i_non_manman AS mothers_name,
    d.j_non_papa AS fathers_name,
    d.k_reskonsab_devan_lalwa AS who_is_your_law_parent,
    d.total,
    d.organisation,
    d.form_link
FROM
    caris_db.dreams_surveys_data d
        LEFT JOIN
    dream_member dm ON dm.case_id = d.case_id
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    dream_group dg ON dg.id = dm.id_group
        LEFT JOIN
    dream_group dpg ON dpg.id = dm.id_parenting_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
"""


ASDATA= read_sql_query(text(query),engine.connect(),parse_dates=True)
ASDATA.id_patient = ASDATA.id_patient.astype(Int32Dtype())
ASDATA.age = ASDATA.age.astype(Int32Dtype())

sdata= read_sql_query(text(query),engine.connect(),parse_dates=True)
sdata.id_patient = sdata.id_patient.astype(Int32Dtype())
sdata.age = sdata.age.astype(Int32Dtype())
SDATA = sdata[sdata.total>=14]

# close the pool of connection
engine.dispose()


False

In [ ]:
datim_served = datim.data_dreams_valid[['id_patient']]
datim_served.shape


(9755, 1)

In [ ]:
datim_served.isna()['id_patient'].unique()


array([False])

In [ ]:
SDATA.columns


Index(['office', 'case_id', 'id_patient', 'code', 'first_name', 'last_name',
       'age', 'age_range', 'ovc_age', 'dob', 'month_in_program_range',
       'interview_date', 'e__telefn', 'adress', 'already_in_a_group',
       'actual_id_group', 'actual_group_name', 'actual_id_parenting_group',
       'actual_parenting_group_name', 'actual_hub', 'actual_id_hub',
       'actual_departement', 'commune', 'commune_section',
       'interviewer_firstname', 'interviewer_lastname', 'interviewer_role',
       'interviewer_other_info', 'interview_location', 'is_your_parent_alive',
       'mothers_name', 'fathers_name', 'who_is_your_law_parent', 'total',
       'organisation', 'form_link'],
      dtype='object')

In [ ]:

SDATA.loc[:,"id_patient"] = SDATA.id_patient.fillna(-1)


/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_20449/3075821563.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_20449/3075821563.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`


In [ ]:
SDATA.isna()['id_patient'].unique()


array([False])

In [ ]:
dreams_groupes = SDATA[['id_patient','actual_id_group','actual_id_hub']]
dreams_groupes 


,id_patient,actual_id_group,actual_id_hub
1,184205,5020.0,346.0
2,162744,4278.0,43.0
3,169576,4551.0,62.0
4,89468,1814.0,147.0
5,129653,3528.0,27.0
...,...,...,...
84621,52192,25.0,10.0
84622,179417,4876.0,337.0
84623,171957,4651.0,268.0
84624,121698,2886.0,190.0


In [ ]:
dreams_groupes.loc[:,'id_patient'] = dreams_groupes.id_patient.astype(int)


/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_20449/1986136293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_20449/1986136293.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`


In [ ]:
actives_groupes = datim_served.merge(dreams_groupes,on='id_patient', how='left')


In [ ]:
actives_groupes


,id_patient,actual_id_group,actual_id_hub
0,75394.0,713.0,100.0
1,75395.0,713.0,100.0
2,75398.0,713.0,100.0
3,75528.0,745.0,104.0
4,75987.0,685.0,47.0
...,...,...,...
9750,174360.0,4731.0,230.0
9751,174358.0,4731.0,230.0
9752,173962.0,2325.0,104.0
9753,173956.0,4059.0,254.0


# Number of groupes

In [ ]:
len(actives_groupes.actual_id_group.unique())


1057

# Number of hubs

In [ ]:
len(actives_groupes.actual_id_hub.unique())


220

# datim results

In [ ]:
datim_served.shape[0]


9755